# Learning how to use Geopandas and Plotly

In this notebook, I experimented with geopandas and plotly to learn how to use each. 

In [ ]:
#Used to install pysal 
pip install pysal

In [ ]:
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd
import geopandas as gp
import shapely.geometry
import json

df = None

#Creates a new Dash app using Plotly inside the notebook
app = Dash()

#Creates the app layout with a title (eGRID Data), a Radio button selection, and an empty graph.
app.layout = [
    html.Div(children='eGRID Data'),
    dcc.RadioItems(options=[
        {'label': html.Div(['Balancing Authority'], style={'color': 'White'}), 'value': 'BA22'}, 
        {'label': html.Div(['State'], style={'color': 'White'}), 'value': 'ST22'}], 
        value='BA22', id='controls-and-radio-item'),
    dcc.Graph(figure={}, id ='controls-and-graph')
]

#Sets up the input and output methods for changing the graph's information
@callback(
    Output(component_id='controls-and-graph', component_property='figure'),
    Input(component_id='controls-and-radio-item', component_property='value')
)

#Updates the graph depending on which button from the Radio section is selected. The information selected in the Radio dial will be displayed in a bar graph. 
def update_graph(col_chosen):
    df = pd.read_excel('Data/egrid2022_data.xlsx', sheet_name=col_chosen)
    column = None
    xAxis = None
    chartHeight = None
    if col_chosen == 'BA22' :
        column = "BA annual CO2 equivalent total output emission rate (lb/MWh)"
        xAxis = "Balancing Authority Name"
        chartHeight = 1000
    elif col_chosen == 'ST22' :
        column = "State annual CO2 equivalent total output emission rate (lb/MWh)"
        xAxis = "State abbreviation"
        chartHeight = 500
    fig = px.histogram(df, x=xAxis, y=column, 
                                  title='Greenhouse Gas emission rate',
                                  height=chartHeight,
                                  labels={'Balancing Authority Name':'Balancing Authority Name', 'BA annual CO2 equivalent total output emission rate (lb/MWh)':'Greenhouse Gas emission rate (lb/MWh)', 'State annual CO2 equivalent total output emission rate (lb/MWh)': 'Greenhouse Gas emission rate (lb/MWh)'},
                                  histfunc='avg'
                                  )
    return fig

#Need to run the app - do not change! 
if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd
import geopandas as gp
import shapely.geometry
import json
from urllib.request import urlopen

#Importing eGRID data to attempt to create a choropleth map 
eGridSpace = gp.read_file("Data/egrid2022_subregions_shapefile.zip")
eGridSpace.to_file('Data/egrid2022_subregions_shapefile.geojson', driver='GeoJSON')

df = pd.read_excel('Data/egrid2022_data.xlsx', sheet_name='SRL22', dtype={"eGRID subregion acronym": str})


In [ ]:
#Attempted to create a choropleth map - does not work yet!
fig = px.choropleth(df, geojson = eGridSpace, locations = "eGRID subregion acronym", color = "eGRID subregion annual CO2 equivalent total output emission rate (lb/MWh)",
                           color_continuous_scale="Viridis")
fig.show()

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gp

#Plots which NERC Coordinator is responsible for which part of the US using Geopandas
nercCoordinators = gp.read_file("Data/NERC_Reliability_Coordinators.shp")

nercCoordinators.plot(column = "NAME", 
               cmap = "gist_ncar", 
               legend = True, 
               legend_kwds={"loc": "center left", "bbox_to_anchor": (1, 0.5)},
              ).set_axis_off()

#Plots the peak load of each energy planning area in the US using Geopandas 
planningAreas = gp.read_file("Data/Planning_Areas.shp")

planningAreas.plot(column = "PEAK_LOAD",
                   cmap = "Reds",
                   legend = True,
                   legend_kwds={'shrink': 0.4, "label": "Peak Load"}
                  ).set_axis_off()